# 分析用データの作成

In [1]:
import pandas as pd
import numpy as np

## BCPデータの整形

In [2]:
# データ読込
path_bcp = '../../data/eol/BCP_data.csv'
bcp = pd.read_csv(path_bcp, header=0, encoding='cp932')

# 列名変更
bcp.columns = [
    "stock_code", "firm_name", "ind_name", "kessan_ymd", "sumbmit_ymd",
    "year", "sogyo_year", "listed_year", "non_listed_year", "document",
    "hit", "BCP_obs"
    ]

# yearのNAを除去
bcp.dropna(subset=['year'], inplace=True)

# 最小年のみ残す
bcp_grouped = bcp.groupby('stock_code')
bcp_grouped = bcp_grouped['year'].min()
bcp_grouped.name = 'BCP_first_year'
bcp = pd.merge(left=bcp, right=bcp_grouped, on='stock_code', how='left')
bcp = bcp.query('year == BCP_first_year')
bcp.drop_duplicates(subset=['stock_code'], inplace=True)

# 列を選択
bcp = bcp[['stock_code', 'year', 'BCP_first_year']]

## 財務データとの結合

In [6]:
path = '../../data/SPEEDA/CompanyList/CompanyListProcessed.csv'
yj = pd.read_csv(path, header=0, encoding='cp932')
yj['stock_code'] = yj['stock_code'].apply(str)
df = pd.merge(left=yj, right=bcp, on=['stock_code', 'year'], how='left')

# 初めてBCPを策定した年以降にBCPを策定していることを表す１を付与する
df_grouped = df.groupby('stock_code')['BCP_first_year'].max()
df_grouped.name = 'fy'
df = pd.merge(left=df, right=df_grouped, on='stock_code', how='left')
bcp_list = [1 if y >= y_bcp else 0 for y, y_bcp in zip(df['year'], df['fy'])]
df['BCP'] = bcp_list
df.drop(columns=['BCP_first_year', 'fy'], inplace=True)
df.dropna(how='any', inplace=True)

path = '../../data/Analysis/analysis_data_b.csv'
df.to_csv(path, encoding='cp932', header=True, index=False)


## 決算期を３月に限定したver

In [7]:
path = '../../data/Analysis/analysis_data_b.csv'
df = pd.read_csv(path, encoding='cp932', header=0)
df = df.query('month == 3')
df.dropna(how='any', inplace=True) # 影響なし
path = '../../data/Analysis/analysis_data_three_b.csv'
df.to_csv(path, encoding='cp932', header=True, index=False)

## サンプル数チェック

In [9]:
path = '../../data/Analysis/analysis_data_b.csv'
df = pd.read_csv(path, encoding='cp932', header=0)
path = '../../data/Analysis/analysis_data_three_b.csv'
df_three = pd.read_csv(path, encoding='cp932', header=0)

def sample_check(df):
    firmnum = df.drop_duplicates(subset=['stock_code'])['stock_code'].count()
    bcp_firmnum = df.query('BCP == 1').drop_duplicates(subset=['stock_code'])['stock_code'].count()
    print(f'企業数：{firmnum}')
    print(f'BCP策定企業数：{bcp_firmnum}')

df_list = [df, df_three]
annot = ['全期間', '3月']
for d, a in zip(df_list, annot):
    print(f'##### {a} #####')
    sample_check(d)

##### 全期間 #####
企業数：2658
BCP策定企業数：907
##### 3月 #####
企業数：1902
BCP策定企業数：718


## データの作成メモ
- outlier: 分布が極端に偏る変数について、上下0.5%ずつの異常値を除去
- year: 年数が飛んでいたり、５年以上の年数が確保できない企業を除去
- standarlized：多くの変数で分布が極端に偏っているため、Yeo-Johnson変換を実施。Box-Cox変換と比べて、負の値を処理できる点から採用した。